对于一元操作, 例如取反和三角函数, 这些ufuncs将会在输出结果中保留索引和列标. 而对于二元操作, 例如加法和乘法, Pandas在把对象传递给ufunc的时候会自动对齐索引.

### Ufuncs: 保留索引  
因为Pandas被设计为可以和Numpy一起使用, 所以任何一个numpy的ufunc都可以在Pandas的Series和DataFrame上使用.

In [6]:
import pandas as pd
import numpy as np

In [12]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [13]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


如果对上面这些对象使用任何一个numpy的ufunc, 那么结果将会是一个保留索引的pandas对象.

In [14]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [15]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


### Ufuncs 对齐索引
对于作用于两个Series或DataFrame对象的二元操作, Pandas在执行操作的过程中将会对齐索引.

#### Series中对齐索引:

In [16]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662, 'California': 423967}, name='area')
area

Alaska        1723337
California     423967
Texas          695662
Name: area, dtype: int64

In [17]:
population = pd.Series({'California': 38332521, 'Texas': 26448193, 'New York': 19651127}, name='population')
population

California    38332521
New York      19651127
Texas         26448193
Name: population, dtype: int64

In [18]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

这个结果包含了两个输入数组索引的Union, 我们也可以在这些索引上使用python的set算术来看看最终的索引:

In [20]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

任何一项如果不在两个数组中同时出现的话, 那么这项的结果就是NaN, 表示该项数据缺失.

In [22]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

如果缺失的数据不想使用NaN, 可以使用适当的对象方法来代替操作符, 这样就可以指定A和B缺失项的填充值了:

In [26]:
C = A.add(B, fill_value = 0)
C

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### DataFrame中的索引对齐
和Series类似, 就是DataFrame中的行索引和列标都会进行对齐动作:

In [27]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=list('AB'))
A

,A,B
0,1,11
1,5,1


In [28]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), columns=list('BAC'))
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [29]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


注意索引对齐的时候跟两个对象中索引的顺序没有关系, 结果的索引是排好序的.  
填充缺失数据:

In [30]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


### Ufuncs: DataFrame和Series之间的操作
当执行DataFrame和Series之间的运算时, 索引和列表都是类似的进行对齐. 就像numpy的二维数组和一维数组间的运算一样.

In [32]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [33]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

根据Numpy的广播规则, 二维数组和一维数组之间的减法是逐行进行的.  
在Pandas里, 类似的操作默认也是行级的:

In [35]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


如果您想按列操作, 可以使用对象方法, 并执行axis关键字:

In [37]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


这些两个元素间的操作, 也都会自动对齐索引:

In [38]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int32

In [40]:
df

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8
2,6,1,3,8


In [41]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN
